In [2]:
import os
import pandas as pd

#folder_path = r"C:\Users\raned\Documents\Github\POSTMODERATION\TrainingData"
folder_path = r"TrainingData"
print("Files in TrainingData folder:")
print(os.listdir(folder_path))

file_path = os.path.join(folder_path, "combined_balanced_dataset_BERT.csv")
training_data_df = pd.read_csv(file_path)

Files in TrainingData folder:
['text.csv', 'training_data.csv', 'HateSpeechDatasetBalanced.csv', 'combined_balanced_dataset_BERT.csv', 'labeled_data.csv']


In [15]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"


In [1]:
# 1. Imports (Make sure you installed required packages already)
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


# 3. Ensure 'Content' is cleaned and valid
training_data_df['Content'] = training_data_df['Content'].fillna('').astype(str)

# 4. Train-validation split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    training_data_df['Content'].tolist(),
    training_data_df['Label'].tolist(),
    test_size=0.2,
    random_state=42
)

# 5. Optional: Use a subset of training data for faster prototyping
train_texts = train_texts[:7000]
train_labels = train_labels[:7000]

# 6. Load DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# 7. Tokenization
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

# 8. PyTorch Dataset wrapper
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(val_encodings, val_labels)

# 9. Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="steps",     
    save_strategy="steps",
    eval_steps=1000,
    save_steps=1000,
    logging_steps=500,
    num_train_epochs=2,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,
    save_total_limit=2,
    logging_dir='./logs',
    fp16=torch.cuda.is_available(),  # Enable mixed precision if using GPU
)

# 🔁 10. Updated Metric Computation — per class
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average=None, labels=[0, 1])
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'precision_not_malicious': precision[0],
        'recall_not_malicious': recall[0],
        'f1_not_malicious': f1[0],
        'precision_malicious': precision[1],
        'recall_malicious': recall[1],
        'f1_malicious': f1[1],
    }

# 11. Trainer Setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# 12. Train
trainer.train()

# 13. Evaluate
results = trainer.evaluate()
print("✅ Evaluation Results:", results)



/opt/anaconda3/envs/ASl-Translator/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
Failed to import transformers.integrations.integration_utils because of the following error (look up to see its traceback):
Failed to import transformers.modeling_tf_utils because of the following error (look up to see its traceback):
Failed to import transformers.data.data_collator because of the following error (look up to see its traceback):
module 'tensorflow' has no attribute 'data'

In [15]:
# 14. Save the trained model and tokenizer
model.save_pretrained("Bert_Model")
tokenizer.save_pretrained("Bert_Model")

print("✅ Model and tokenizer saved to 'Bert_Model/'")



✅ Model and tokenizer saved to 'Bert_Model/'


In [16]:
import os

print("Current working directory:", os.getcwd())
print("Saved files in 'Bert_Model':", os.listdir("Bert_Model"))



Current working directory: C:\Users\raned
Saved files in 'Bert_Model': ['config.json', 'model.safetensors', 'special_tokens_map.json', 'tokenizer_config.json', 'vocab.txt']


In [6]:

# 1. Define new texts
new_texts = [
    "Fuck this place.", 
    "What a beautiful day, feeling grateful!",
    "I hate it here.",
    "You're a piece of shit",
    "Dumb ass bitch",
    "you’re such a dumbass 🤡 nobody wants you around 💩", #testing malicious tweets with emojis
    "go back to your country 🖕",
    "You're such a clown 🤡 lol",
    "🖕🤡🖕"
]

# 2. Tokenize the new content
new_encodings = tokenizer(new_texts, truncation=True, padding=True, max_length=128, return_tensors="pt")

# 3. Run the model in evaluation mode (no gradients)
model.eval()
with torch.no_grad():
    outputs = model(**new_encodings)
    predictions = torch.argmax(outputs.logits, dim=1)


# 4. Print results with readable labels
labels = ['Not Malicious', 'Malicious']
for text, pred in zip(new_texts, predictions):
    print(f"Text: {text}\nPrediction: {labels[pred]}\n")

Text: Fuck this place.
Prediction: Malicious

Text: What a beautiful day, feeling grateful!
Prediction: Not Malicious

Text: I hate it here.
Prediction: Not Malicious

Text: You're a piece of shit
Prediction: Malicious

Text: Dumb ass bitch
Prediction: Malicious

Text: you’re such a dumbass 🤡 nobody wants you around 💩
Prediction: Malicious

Text: go back to your country 🖕
Prediction: Not Malicious

Text: You're such a clown 🤡 lol
Prediction: Not Malicious

Text: 🖕🤡🖕
Prediction: Not Malicious

